# RDFlib Graphs

(C) 2023-2024 by [Damir Cavar](http://damir.cavar.com/)

This notebook provides examples how to use [RDFlib](https://rdflib.readthedocs.io/) to read and write [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework), [Turtle](https://en.wikipedia.org/wiki/Turtle_(syntax)), and other serialized graph representation.

**Download:** This and various other Jupyter notebooks are available from my [GitHub repo](https://github.com/dcavar/python-tutorial-notebooks).

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

**Prerequisites:**

In [ ]:
!pip install -U rdflib

This is a research seminar related notebook created for the seminar on *[Knowledge Graphs, Large Language Models (LLM), and Graph-based Reasoning](http://damir.cavar.me/l715-23/)* taught at [Indiana University at Bloomington](https://bloomington.iu.edu/) in Fall 2023.

We import the [RDFlib](https://rdflib.readthedocs.io/) components that we intend to use:

In [53]:
from rdflib.namespace import RDF, RDFS, SKOS, OWL, DC, DCTERMS, XSD, TIME, NamespaceManager
from rdflib import Graph, URIRef, Literal, Namespace
from datetime import datetime

We want to use specific URIs in the graph, as for example the [Ontology of Linguistic Annotation](https://acoli-repo.github.io/olia/) ([OLiA](https://acoli-repo.github.io/olia/)) and the [Vocabulary for Attaching Essential Metadata](https://lov.linkeddata.es/dataset/lov/vocabs/vaem) ([vaem](https://lov.linkeddata.es/dataset/lov/vocabs/vaem)):

In [54]:
default_prefix = r"http://www.iu.edu/kg/ontology/Example#"
default_uri    = URIRef(r"http://www.iu.edu/kg/ontology/")
default_ns     = Namespace(default_prefix)
olia_ns        = Namespace(r"http://purl.org/olia/olia.owl#")
vaem_ns        = Namespace(r"http://www.linkedmodel.org/schema/vaem#")

Creating a graph and defining the namespace for some relevant ontologies as defined above.

In [55]:
graph = Graph()
namespace_manager = NamespaceManager(graph)
namespace_manager.bind("olia", olia_ns, override=True)
namespace_manager.bind("vaem", vaem_ns, override=True)
namespace_manager.bind("iu", default_ns, override=True)
graph.namespace_manager = namespace_manager

In the following code block we are adding triples to the graph:

In [56]:
graph.add( (default_uri, RDF.type,           OWL.Ontology) )
graph.add( (default_uri, RDFS.label,         Literal("Example Ontology", lang='en')) )
graph.add( (default_uri, RDFS.label,         Literal("Beispielontology", lang='de')) )
graph.add( (default_uri, SKOS.prefLabel,     Literal("Example Ontology")) )
graph.add( (default_uri, TIME.year,          Literal("2023")) )
graph.add( (default_uri, vaem_ns.acronym,    Literal("EO")) )
graph.add( (default_uri, DC.creator,         Literal("Damir Cavar")) )
graph.add( (default_uri, vaem_ns.hasOwner,   Literal("Natural Language Processing Lab", lang='en')) )
graph.add( (default_uri, vaem_ns.hasSteward, Literal("Natural Language Processing Lab")) )
dt = datetime.now()
graph.add( (default_uri, vaem_ns.publishedDate, Literal(dt.strftime("%Y-%m-%dT%H:%M:%SZ"))) )

<Graph identifier=Nbbdc38a39763458bb883e149f684a691 (<class 'rdflib.graph.Graph'>)>

In the following we add a concept to the graph:

In [57]:
concept_uri = URIRef(default_prefix + r"TestConcept")
graph.add( (concept_uri, RDFS.label, Literal("Test Concept", lang='en')) )
graph.add( (concept_uri, RDFS.label, Literal("Testkonzept", lang='de')) )

<Graph identifier=Nbbdc38a39763458bb883e149f684a691 (<class 'rdflib.graph.Graph'>)>

We can insert data into the graph using [SPARQL](https://www.w3.org/TR/sparql11-query/) as well:

In [58]:
query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX vaem: <http://www.linkedmodel.org/schema/vaem#>
PREFIX iu: <http://www.iu.edu/kg/ontology/Example#>
INSERT DATA {{
    <{default_prefix + r'ExampleConcept'}> rdfs:label "Example Concept"@en,
                                                      "Beispielkonzept"@de .
    <{default_prefix + r'ExampleConcept'}> vaem:acronym "EC" ;
                                           vaem:abbreviation "ExCo" .
}}"""
graph.update(query)

Writing out the graph to a [Turtle](https://en.wikipedia.org/wiki/Turtle_(syntax)) file:

In [59]:
graph.serialize(destination="test.ttl", format="turtle", encoding="utf-8")

<Graph identifier=Nbbdc38a39763458bb883e149f684a691 (<class 'rdflib.graph.Graph'>)>

Writing out the graph to an [N-Triples](https://en.wikipedia.org/wiki/N-Triples) file:

In [60]:
graph.serialize(destination="test.nt", format="nt", encoding="utf-8")

<Graph identifier=Nbbdc38a39763458bb883e149f684a691 (<class 'rdflib.graph.Graph'>)>

## Graph Queries

In the following code we are querying all triples with a specific pattern and using the subject and object elements only. Note, since the elements are `URLib` objects and need to be converted to strings using the `toPython()` method.

In [61]:
res = { (s.toPython(), o.toPython()) for s, _, o in graph.triples( (None, RDFS.label, None) ) }
for r in res:
    print(r)

('http://www.iu.edu/kg/ontology/', 'Beispielontology')
('http://www.iu.edu/kg/ontology/Example#TestConcept', 'Testkonzept')
('http://www.iu.edu/kg/ontology/', 'Example Ontology')
('http://www.iu.edu/kg/ontology/Example#ExampleConcept', 'Example Concept')
('http://www.iu.edu/kg/ontology/Example#ExampleConcept', 'Beispielkonzept')
('http://www.iu.edu/kg/ontology/Example#TestConcept', 'Test Concept')


In the following query we are looking for all triples that match with the predicate and object, resulting in a list of subjects:

In [62]:
res = { s.toPython() for s in graph.subjects( RDFS.label, Literal('Example Concept', lang="en")) }
for r in res:
    print(r)

http://www.iu.edu/kg/ontology/Example#ExampleConcept


In the following we are querying all the objects that are elements of triples with the specific subject and predicate:

In [63]:
res = { s.toPython() for s in graph.objects(URIRef(r"http://www.iu.edu/kg/ontology/Example#ExampleConcept"), RDFS.label) }
for r in res:
    print(r)

Beispielkonzept
Example Concept


The same queries we can formulate in SPARQL:

In [64]:
qres = graph.query(
    """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?label
        WHERE {
            ?s rdfs:label ?label
            FILTER langMatches( lang(?label), "en" )
        }"""
)
for row in qres:
    print(row.label)

Example Ontology
Test Concept
Example Concept


(C) 2023-2024 by [Damir Cavar](http://damir.cavar.com/)